In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
import time
import numpy as np
import json
import random

from model import FewShotModel
from dataloader import Dataset

In [10]:
'''
Setting Up argument parser as a class for this example. While running the actual main.py file,
argparse can be directly utilized for command line argumnts
'''
class Args:
    dataset_name = "TRIANGLES"
    device = 0
    num_layers = 5
    num_mlp_layers = 2
    hidden_dim = 128
    final_dropout = 0.5
    graph_pooling_type = "sum"
    neighbor_pooling_type = "sum"
    learn_eps = True
    degree_as_tag = True
    
    num_gat_layers = 2
    gat_out_dim = 128
    gat_dropout = 0.5
    gat_heads = 2
    gat_leaky_slope = 0.1
    gat_concat = 0
    
    is_train = 1
    is_test = 1
    
    batch_size = 64
    iters_per_epoch = 10
    epochs = 50
    lr = 0.001
    weight_decay = 1e-7
    save_model_after = 1
    n_shot = 20
    model_runs = 1
    knn_value = 2
    train_clusters = 3
    num_inference_graphs = 500
    
    fine_tune_lr = 0.001
    fine_tune_weight_decay = 1e-7
    fine_tune_epochs = 20
    fine_tune_iters_per_epoch = 10
    fine_tune_batch_size = 500
    
    fine_tune_save_model_after = 1
    num_testing_runs = 10

In [11]:
args = Args()

knn_params = {"initial": args.knn_value}
gat_layer_params = {}

for i in range(args.num_gat_layers):
    knn_params[i] = args.knn_value

    gat_layer_params[i] = {}
    if i == 0:
        gat_layer_params[i]["in_channels"] = args.hidden_dim * (args.num_layers-1)
        gat_layer_params[i]["out_channels"] = args.gat_out_dim

    else:
        if args.gat_concat == 1:
            gat_layer_params[i]["in_channels"] = gat_layer_params[i-1]["out_channels"] * \
                                                 gat_layer_params[i-1]["heads"]
        else:
            gat_layer_params[i]["in_channels"] = gat_layer_params[i-1]["out_channels"]
        gat_layer_params[i]["out_channels"] = args.gat_out_dim

    if args.gat_concat == 1:
        gat_layer_params[i]["concat"] = True
    else:
        gat_layer_params[i]["concat"] = False
    gat_layer_params[i]["heads"] = args.gat_heads
    gat_layer_params[i]["leaky_slope"] = args.gat_leaky_slope
    gat_layer_params[i]["dropout"] = args.gat_dropout

In [12]:
device = torch.device("cuda:" + str(0)) if torch.cuda.is_available() else torch.device("cpu")

In [15]:
'''
Creating Instance of the model here
'''

overall_acc = []
for runs in range(args.model_runs):
    print("\nCurrent run = ", runs+1, "\n")
    main_start_time = time.time()
    
    dataset = Dataset(args.dataset_name, args)
    dataset.segregate(args, runs)
    
    if args.is_train == 1:
        train_start_time = time.time()

        train_start_time = time.time()
        few_shot_model = FewShotModel(args.num_layers, args.num_mlp_layers, 
                dataset.train_graphs[0].node_features.shape[1], args.hidden_dim, 
                args.train_clusters,
                args.final_dropout, args.learn_eps, args.graph_pooling_type, 
                args.neighbor_pooling_type, device, args.num_gat_layers, gat_layer_params, 
                knn_params, dataset, "train").to(device)

        few_shot_model.train_(args, dataset)
        print("\nRemaining Training phase finished in ", time.time() - train_start_time, " seconds\n")
        
    if args.is_test == 1:
        print("\nFine tuning GIN for super-class classification on test classes...\n")
        test_start_time = time.time()

        few_shot_model = FewShotModel(args.num_layers, args.num_mlp_layers, 
                dataset.train_graphs[0].node_features.shape[1], args.hidden_dim, 
                args.train_clusters,
                args.final_dropout, args.learn_eps, args.graph_pooling_type, 
                args.neighbor_pooling_type, device, args.num_gat_layers, gat_layer_params, 
                knn_params, dataset, "test").to(device)

        few_shot_model.test_fine_tuning(args, dataset, device)
        print("\nFine tuning finished in ", time.time() - test_start_time, " seconds\n")
        current_run_acc = few_shot_model.test(args, dataset, device)
        overall_acc.append(current_run_acc)
        
    print("\nTotal time taken for 1 complete run = ", time.time() - main_start_time, " seconds\n")
    
print("\nAverage acc over the model runs = ", np.mean(overall_acc), " , std = ", np.std(overall_acc))


Current run =  1 


Number of super clusters =  3 

loading data
# classes: 10
# maximum node tag: 10
# data: 2010 

Loading class splits
Number of training graphs =  1126
Number of validation graphs =  281
Number of total testing graphs =  603 


Number of test fine tuning graphs =  60
Number of test evaluation graphs =  543 

average loss for training epoch  0  =  1.8963455200195312
average loss for training epoch  1  =  1.793415641784668
Saving Main Model in training phase
average loss for training epoch  2  =  1.673684310913086
Saving Main Model in training phase
average loss for training epoch  3  =  1.5658200263977051
Saving Main Model in training phase
average loss for training epoch  4  =  1.4544219017028808
Saving Main Model in training phase
average loss for training epoch  5  =  1.4357110023498536
Saving Main Model in training phase
average loss for training epoch  6  =  1.3649810791015624
Saving Main Model in training phase
average loss for training epoch  7  =  1.35750064